# Joint Named Entity Recognition and Relation Extraction using DSPY

## Default definition for model usage and imports

In [1]:
from IPython.display import Markdown, display
from collections import Counter
from devtools import pprint
from dotenv import load_dotenv
from dspy.functional import TypedPredictor
from dspy.teleprompt import LabeledFewShot
from pydantic import BaseModel, Field
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from tqdm import tqdm
from typing import List, Optional, Union
import ast
import dspy
import json
import matplotlib.pyplot as plt
import nest_asyncio
import numpy as np
import os
import pandas as pd
import pickle
import seaborn as sns

assert load_dotenv("/work3/s174159/LLM_Thesis/prompt/.env") == True

In [2]:
API = os.getenv('OPENAI')
HF = os.getenv("HF")

In [3]:
gpt = dspy.OpenAI(model='gpt-4o-mini', max_tokens=2048, api_key=API)
gpt4 = dspy.OpenAI(model='gpt-4', max_tokens=1024, api_key= API)
gpt3 = dspy.OpenAI(model='gpt-3.5-turbo', max_tokens=2048, api_key=API)


# Set default model
dspy.settings.configure(lm=gpt4)

## Create sample dataset and test data

In [4]:
LOAD_FROM_FILES = True

In [5]:
SAMPLE_SIZE = 50

In [ ]:
if LOAD_FROM_FILES:
    test_input = pd.read_json("/work3/s174159/LLM_Thesis/data/Final/Combined/test_sampled.json")
    dev_input = pd.read_json("/work3/s174159/LLM_Thesis/data/Final/Combined/dev_sampled.json")
    sampled_df = pd.read_json("/work3/s174159/LLM_Thesis/data/Final/Combined/train_sampled.json")

df1 = pd.read_json('/work3/s174159/LLM_Thesis/data/Final/Combined/train_annotated.json')
#df2 = pd.read_json("/work3/s174159/LLM_Thesis/data/No_Constraints/Data_Finalized/Combined/train_distant.json")
dataframes = [df1]
df = pd.concat(dataframes)
df['org_dataset'].value_counts()

In [7]:
def sample_equal_per_class(df, sample_size, column_name):
    unique_classes = df[column_name].unique()
    n_samples_per_class = sample_size // len(unique_classes)  # Equal samples per class

    sampled_df = pd.DataFrame()

    # Sample equal number of samples from each class
    for category in unique_classes:
        sampled_df = pd.concat([sampled_df, df[df[column_name] == category].sample(n=n_samples_per_class, random_state=42)])

    return sampled_df.sample(frac=1).reset_index(drop=True)


In [8]:
if not LOAD_FROM_FILES:
    sampled_df = sample_equal_per_class(df, SAMPLE_SIZE, 'org_dataset')

In [ ]:
original_distribution = df['org_dataset'].value_counts(normalize=True) * 100
sampled_distribution = sampled_df['org_dataset'].value_counts(normalize=True) * 100


distribution_df = pd.DataFrame({'Original': original_distribution, 'Sampled': sampled_distribution}).sort_index()

plt.figure(figsize=(10, 6))
distribution_df.plot(kind='bar', width=0.8)
plt.title('Distribution of classes in the original and sampled dataset')
plt.ylabel('Percentage')
plt.xlabel('Class')
plt.xticks(rotation=0)
plt.show()

In [10]:
def prepare_dataset(df):
    # Prepare the DSPY dataset
    dataset = [None] * len(df.values)
    for i,(org_dataset, title, domains, sents, vertexSet, labels, original_file_path) in tqdm(enumerate(df.values), total=len(df), desc="Creating dataset - Processing row"):
        # Convert JSON strings to Python objects
        sents = (sents)
        vertexSet = (vertexSet)
        labels = (labels)
        
        entity_types = list(set([entity['type'] for entity_list in vertexSet for entity in entity_list]))
        relation_types = list(set([relation['r'] for relation in labels]))
        
        # Create a DSPY example and append to the dataset
        example = dspy.Example(
            sentences=str(sents), 
            entities=vertexSet, 
            relations=labels
        ).with_inputs('sentences', 'entity_types', 'relation_types')

        # Attach entity_types and relation_types to the example
        example['entity_types'] = str(entity_types)
        example['relation_types'] = str(relation_types)
        
        dataset[i] = example

    # Now the dataset is ready for use with DSPY
    print(f"Total examples created: {len(dataset)}")

    # Check one example to see the structure
    print(dataset[0])
    return dataset


In [ ]:
dataset = prepare_dataset(sampled_df)
dev_data = prepare_dataset(dev_input)
test_data = prepare_dataset(test_input)

In [12]:
if not LOAD_FROM_FILES:
    dev_input = pd.read_json("/work3/s174159/LLM_Thesis/data/Final/Combined/dev.json")
    dev_input = sample_equal_per_class(dev_input, SAMPLE_SIZE, 'org_dataset')
    dev_data = prepare_dataset(dev_input)

In [13]:
if not LOAD_FROM_FILES:
    test_input = pd.read_json("/work3/s174159/LLM_Thesis/data/Final/Combined/test.json")
    test_input = sample_equal_per_class(test_input, SAMPLE_SIZE, 'org_dataset')
    test_input.to_json("/work3/s174159/LLM_Thesis/data/Final/Combined/test_sampled.json")
    dev_input.to_json("/work3/s174159/LLM_Thesis/data/Final/Combined/dev_sampled.json")
    sampled_df.to_json("/work3/s174159/LLM_Thesis/data/Final/Combined/train_sampled.json")
    test_data = prepare_dataset(test_input)

## Define DSPY Signatures

In [14]:
class Entity_extractor(dspy.Signature):
    """Extract entities from a provided text based on specified entity types.
    The output is a list of tuples in the format (entity, type, location, evidence), where:
    - 'entity' is the extracted term,
    - 'type' is its classification (e.g., Person, Location),
    - 'location' is the index range within the text where the entity is found, represented as a list [Start, End],
    - 'evidence' is a list of indices indicating the specific sublists (within the list of lists) that contain the entity."""
    
    sentences = dspy.InputField(description="A list of lists, where each inner list contains text segments to extract entities from.", type=str, required=True)
    entity_types = dspy.InputField(description="The types of entities to extract (e.g., Person, Location, Organization).", type=str, required=True)
    answer = dspy.OutputField(description="A string representing the list of extracted entities in the format (entity, type, location, evidence).", type=str)



class TupleToJson(dspy.Signature):
    """Convert extracted entity tuples into JSON format.
    The output is a list of dictionaries, each containing:
    - 'entity': the extracted term,
    - 'type': its classification (e.g., Person, Location),
    - 'location': the index range within the text where the entity is found,
    - 'evidence': a list of indices indicating the sublists containing the entity.
    The output should be formatted as a JSON list."""
    
    text = dspy.InputField(description="Input text containing extracted entities in tuple format, i.e., (entity, type, location, evidence).", type=str, required=True)
    answer = dspy.OutputField(description="JSON-formatted output as a list of dictionaries. Remember to include [ ] to mark the list", type=str)


    
class RelationExtractor(dspy.Signature):
    """
    Given the text in form of Python lists of sentences, a list of dictionaries with entities and a list of relation types find the relations between the entities.
    The output should be  a list of tuples in the format (entity1, entity2, relation), where:
    - 'entity1' and 'entity2' are the names or terms representing the two entities involved,
    - 'relation' is the specific relationship type between them, as defined in the provided relation types. Use the exact types as defined in the input field, relation_types. 
    This format captures the relationships directly as they appear in the text."""
    
    entities = dspy.InputField(description="A list of entities in dictionary format [{'entity': name, 'type': entity_type, 'location': index}].", type=str, required=True)
    sentences = dspy.InputField(description="The original text containing the entities to identify relationships between them.", type=str, required=True)
    relation_types = dspy.InputField(description="Optional: Specific types of relationships to extract between the entities (e.g., Parent-Child, Colleague).", type=str, required=False)
    answer = dspy.OutputField(description="A string representing the list of identified relations in the format (entity1, entity2, relation).", type=str)

class TupleToJsonRelation(dspy.Signature):
    """Convert text into a structured JSON format representing relationships between entities.
    The output should be a list of dictionaries, each containing:
    - 'entity1': the first entity in the relationship,
    - 'entity2': the second entity in the relationship,
    - 'relation': the type of relationship between the entities.
    The output should be formatted as a JSON list."""
    
    text = dspy.InputField(description="Input text containing entity relationships described in natural language.", type=str, required=True)
    answer = dspy.OutputField(description="JSON-formatted output as a list of dictionaries, where each dictionary contains 'entity1', 'entity2', and 'relation type'. Do not include ```json tags. But only the json list itself", type=str)



In [15]:
class CoTRelationExtraction(dspy.Module):
    def __init__(self):
        super().__init__()
        self.entity_extraction = dspy.ChainOfThought(Entity_extractor)
        self.tuple_to_json = dspy.ChainOfThought(TupleToJson)
        self.relation_extraction = dspy.ChainOfThought(RelationExtractor)
        self.tuple_to_json_relation = dspy.ChainOfThought(TupleToJsonRelation)
        
        
    
    def forward(self, sentences, entity_types, relation_types):
        entities = self.entity_extraction(sentences=sentences, entity_types=entity_types)
        entities = entities.answer.split("---")[0]
        entities = self.tuple_to_json(text=entities)
        entities = str(entities.answer)
        relations = self.relation_extraction(entities=entities, sentences=sentences, relation_types = relation_types)
        # print(relations)
        relations = relations.answer.split("---")[0]
        # print(relations)
        relations = self.tuple_to_json_relation(text=relations)
        relations = str(relations.answer)
        # print(relations)
        return str(relations)
    

In [16]:
rel_extractor = CoTRelationExtraction()

In [17]:
#scores_rels = []
#results = []


def _parse_input(input_data):
    if isinstance(input_data, str):
        try:
            # Try to parse as JSON
            return json.loads(input_data)
        except json.JSONDecodeError:
            try:
                # Fallback to ast.literal_eval
                return ast.literal_eval(input_data)
            except (ValueError, SyntaxError):
                # Return None if both parsing attempts fail
                return None
    elif isinstance(input_data, dict):
        # Wrap dict in a list
        return [input_data]
    return input_data


def combined_relation_f1_score(expected, predicted, trace=None):
    """
    The following is only used to establish a baseline and thus re-trace the steps!
    """

    # Parse expected and predicted inputs
    expected = _parse_input(expected)
    predicted = _parse_input(predicted)

    if expected is None:
        return (
            {
                "micro_f1": 0,
                "micro_precision": 0,
                "micro_recall": 0,
                "macro_f1": 0,
                "macro_precision": 0,
                "macro_recall": 0,
                "accuracy": 0,
                "class_metrics": {},
            },
            {
                "labels": [],
                "predictions": [],
            },
        )

    # Handle case where predicted or expected is None or empty
    if len(predicted) == 0 and len(expected) == 0:
        # Return zero-valued metrics
        return (
            {
                "micro_f1": 0,
                "micro_precision": 0,
                "micro_recall": 0,
                "macro_f1": 0,
                "macro_precision": 0,
                "macro_recall": 0,
                "accuracy": 0,
                "class_metrics": {},
            },
            {"labels": [], "predictions": []},
        )
    elif len(expected) == 0:
        return (
            {
                "micro_f1": 0,
                "micro_precision": 0,
                "micro_recall": 0,
                "macro_f1": 0,
                "macro_precision": 0,
                "macro_recall": 0,
                "accuracy": 0,
                "class_metrics": {},
            },
            {
                "labels": [],
                "predictions": [
                    {
                        "head": rel["entity1"],
                        "tail": rel["entity2"],
                        "type": rel["relation"],
                        "correct": False,
                    }
                    for rel in predicted
                ],
            },
        )

    expected_relations = set()

    # Extract expected relations from entities and relations
    entities = expected.get("entities", [])
    relations = expected.get("relations", [])

    flat_entities = [entity for sublist in entities for entity in sublist]
    for rel in relations:
        entity1_name = flat_entities[rel["h"]]["name"]
        entity2_name = flat_entities[rel["t"]]["name"]
        relation_type = rel["r"]
        expected_relations.add((entity1_name, entity2_name, relation_type))

    if predicted is None or len(predicted) == 0:
        return (
            {
                "micro_f1": 0,
                "micro_precision": 0,
                "micro_recall": 0,
                "macro_f1": 0,
                "macro_precision": 0,
                "macro_recall": 0,
                "accuracy": 0,
                "class_metrics": {},
            },
            {
                "labels": [
                    {"head": head, "tail": tail, "type": _type, "correct": False}
                    for (head, tail, _type) in expected_relations
                ],
                "predictions": [],
            },
        )

    # Extract predicted relations
    predicted_relations = {
        (rel["entity1"], rel["entity2"], rel["relation"]) for rel in predicted
    }

    # Create all possible labels for binary classification (1 if relation is in expected, 0 otherwise)
    all_relations = list(expected_relations | predicted_relations)

    y_true = [1 if rel in expected_relations else 0 for rel in all_relations]
    y_pred = [1 if rel in predicted_relations else 0 for rel in all_relations]

    # Check if y_true and y_pred are non-empty before calculating metrics
    if len(y_true) == 0 or len(y_pred) == 0:
        return (
            {
                "micro_f1": 0,
                "micro_precision": 0,
                "micro_recall": 0,
                "macro_f1": 0,
                "macro_precision": 0,
                "macro_recall": 0,
                "accuracy": 0,
                "class_metrics": {},
            },
            {
                "labels": [
                    {
                        "head": head,
                        "tail": tail,
                        "type": _type,
                        "correct": (head, tail, _type) in predicted_relations,
                    }
                    for (head, tail, _type) in expected_relations
                ],
                "predictions": [
                    {
                        "head": head,
                        "tail": tail,
                        "type": _type,
                        "correct": (head, tail, _type) in expected_relations,
                    }
                    for (head, tail, _type) in predicted_relations
                ],
            },
        )

    # Calculate metrics
    metrics = {}

    # Micro F1, precision, recall
    metrics["micro_f1"] = f1_score(y_true, y_pred, average="micro", zero_division=0)
    metrics["micro_precision"] = precision_score(
        y_true, y_pred, average="micro", zero_division=0
    )
    metrics["micro_recall"] = recall_score(
        y_true, y_pred, average="micro", zero_division=0
    )

    # Macro F1, precision, recall
    metrics["macro_f1"] = f1_score(y_true, y_pred, average="macro", zero_division=0)
    metrics["macro_precision"] = precision_score(
        y_true, y_pred, average="macro", zero_division=0
    )
    metrics["macro_recall"] = recall_score(
        y_true, y_pred, average="macro", zero_division=0
    )

    # Per-class metrics (F1, precision, recall)
    unique_classes = sorted(
        set([rel[2] for rel in all_relations])
    )  # Classes are relation types
    class_metrics = {cls: {} for cls in unique_classes}

    for i, cls in enumerate(unique_classes):
        y_true_class = [
            1 if rel[2] == cls and rel in expected_relations else 0
            for rel in all_relations
        ]
        y_pred_class = [
            1 if rel[2] == cls and rel in predicted_relations else 0
            for rel in all_relations
        ]

        class_metrics[cls]["f1"] = f1_score(y_true_class, y_pred_class, zero_division=0)
        class_metrics[cls]["precision"] = precision_score(
            y_true_class, y_pred_class, zero_division=0
        )
        class_metrics[cls]["recall"] = recall_score(
            y_true_class, y_pred_class, zero_division=0
        )
        class_metrics[cls]["accuracy"] = accuracy_score(y_true_class, y_pred_class)

    metrics["class_metrics"] = class_metrics

    # Overall accuracy
    metrics["accuracy"] = accuracy_score(y_true, y_pred)
    #scores_rels.append(metrics)  # For the parallelization
    return (
        metrics,
        {
            "labels": [
                {
                    "head": head,
                    "tail": tail,
                    "type": _type,
                    "correct": (head, tail, _type) in predicted_relations,
                }
                for (head, tail, _type) in expected_relations
            ],
            "predictions": [
                {
                    "head": head,
                    "tail": tail,
                    "type": _type,
                    "correct": (head, tail, _type) in expected_relations,
                }
                for (head, tail, _type) in predicted_relations
            ],
        },
    )

### Check of the samples

In [ ]:
dataset[0].inputs()['relation_types']

In [ ]:
relations_type_counter = Counter()

for sample in dataset: 
    relation_types = sample.inputs()['relation_types']
    relations_type_counter.update(ast.literal_eval(relation_types))

relation_types_count = dict(relations_type_counter)
print(relation_types_count)

In [ ]:
len(set(list(relation_types_count.keys())))

In [21]:
original_test_set_types = set(list(relation_types_count.keys()))

In [ ]:
# Convert dictionary keys and values into separate lists for plotting
x_values = list(relation_types_count.keys())
y_values = list(relation_types_count.values())

# Set the style for the plot
plt.figure(figsize=(16, 6))
plt.bar(x_values, y_values, color="skyblue")

# Add title and labels
plt.title('Relation Type Frequency', fontsize=16)
plt.xlabel('Relation Types', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.xticks(rotation=45, ha="right", fontsize = 8)  # Rotate x labels for better readability

# Display the plot
plt.tight_layout()
plt.show()

In [ ]:
dataset[0]

In [ ]:
scores_rels = [None] * 3
predictions = [None] * 3
label_predictions = [None] * 3
incorrect_format = 0

for i,sam in tqdm(enumerate(dataset[3:6]), desc="Evaluating test data -- Setting baseline"):
    pred = rel_extractor(**sam.inputs())
    predictions[i] = pred
    scores_rels[i],label_predictions[i] = combined_relation_f1_score(sam, pred)
    incorrect_format += len(label_predictions[i]["predictions"]) == 0
# return scores_rels



In [ ]:
def niels_parse_input(input_data):
    if isinstance(input_data, str):
        print("a")
        try:
            # Try to parse as JSON
            return json.loads(input_data)
        except json.JSONDecodeError:
            try:
                # Fallback to ast.literal_eval
                return ast.literal_eval(input_data)
            except (ValueError, SyntaxError):
                # Return None if both parsing attempts fail
                return None
    elif isinstance(input_data, dict):
        print("b")
        # Wrap dict in a list
        return [input_data]

    print("c")
    return input_data

# for pred in predictions:
#     print(pred)
#     print(niels_parse_input(pred))
#     # print()
# for sam in dataset[:3]:
#     print(niels_parse_input(sam))

# combined_relation_f1_score(sam,"""[
#     {"entity1": "FKT", "entity2": "multi-class problem", "relation": "Used for"},
#     {"entity1": "Kernel Fukunaga-Koontz Transform", "entity2": "face recognition applications", "relation": "Used for"},
#     {"entity1": "Fukunaga-Koontz Transform", "entity2": "FKT","relation": "Conjunction"}
# ]""")

# label_predictions
incorrect_format

In [26]:
def get_score(program, test_data):
    scores_rels = [None] * len(test_data)
    label_predictions = []
    incorrect_format = [] 

    for i,sam in tqdm(enumerate(test_data), desc="Evaluating test data -- Setting baseline"):
        pred = program(**sam.inputs())
        scores_rels[i],lp = combined_relation_f1_score(sam, pred)
        if len(lp["predictions"]) == 0:
            incorrect_format.append(lp)
        else:
            label_predictions.append(lp)
    return scores_rels,label_predictions,incorrect_format

## Baseline scoring

In [ ]:
scores_rels,label_predictions,incorrect_format = get_score(rel_extractor,test_data)

In [28]:
def sort_data(scores_rels):
    # Create a list of dictionaries for the general metrics
    general_metrics_data = [None] * len(scores_rels)
    for i,item in enumerate(scores_rels):
        general_metrics_data[i] = ({
            'micro_f1': item['micro_f1'],
            'micro_precision': item['micro_precision'],
            'micro_recall': item['micro_recall'],
            'macro_f1': item['macro_f1'],
            'macro_precision': item['macro_precision'],
            'macro_recall': item['macro_recall'],
            'accuracy': item['accuracy']
        })

    # Convert to a DataFrame
    general_metrics_df = pd.DataFrame(general_metrics_data)


        # Create a list for class-specific metrics
    class_metrics_data = []
    for item in scores_rels:
        for class_name, metrics in item['class_metrics'].items():
            class_metrics_data.append({
                'class': class_name,
                'f1': metrics['f1'],
                'precision': metrics['precision'],
                'recall': metrics['recall'],
                'accuracy': metrics['accuracy']
            })

    # Convert to a DataFrame
    class_metrics_df = pd.DataFrame(class_metrics_data)
    return general_metrics_df, class_metrics_df



In [29]:
general_metrics_df, class_metrics_df = sort_data(scores_rels)

In [ ]:
general_metrics_df.describe()

In [31]:
def plot_metrics(general_metrics_df, class_metrics_df):
    # Set the style for the plots
    sns.set(style="whitegrid")

    # Plot for General Metrics
    fig, axes = plt.subplots(2, 3, figsize=(30, 10))
    fig.suptitle('General Metrics over Experiments', fontsize=16)
    
    metrics = ['micro_f1', 'micro_precision', 'micro_recall', 'macro_f1', 'macro_precision', 'macro_recall']
    
    for idx, metric in enumerate(metrics):
        row = idx // 3
        col = idx % 3
        axes[row, col].plot(general_metrics_df.index, general_metrics_df[metric], marker='o')
        axes[row, col].set_title(metric)
        axes[row, col].set_xlabel('Experiment')
        axes[row, col].set_ylabel(metric)
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

    # Plot for Class-Specific Metrics
    unique_classes = class_metrics_df['class'].unique()

    for metric in ['f1', 'precision', 'recall', 'accuracy']:
        plt.figure(figsize=(30, 6))
        sns.barplot(data=class_metrics_df, x='class', y=metric)
        plt.title(f'Class-Specific {metric.capitalize()}')
        plt.xlabel('Class')
        plt.ylabel(metric.capitalize())
        plt.xticks(rotation=45, ha="right")
        plt.tight_layout()
        plt.show()

In [32]:
def plot_metrics_with_class_occurrences_and_f1(general_metrics_df, class_metrics_df, class_occurrences_dict):
    # Set the style for the plots
    sns.set(style="whitegrid")

    # Plot for General Metrics
    fig, axes = plt.subplots(2, 3, figsize=(30, 10))
    fig.suptitle('General Metrics over Experiments', fontsize=16)
    
    metrics = ['micro_f1', 'micro_precision', 'micro_recall', 'macro_f1', 'macro_precision', 'macro_recall']
    
    for idx, metric in enumerate(metrics):
        row = idx // 3
        col = idx % 3
        axes[row, col].plot(general_metrics_df.index, general_metrics_df[metric], marker='o')
        axes[row, col].set_title(metric)
        axes[row, col].set_xlabel('Experiment')
        axes[row, col].set_ylabel(metric)
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

    # Plot for Class-Specific Metrics
    unique_classes = class_metrics_df['class'].unique()

    for metric in ['f1', 'precision', 'recall', 'accuracy']:
        plt.figure(figsize=(30, 6))
        sns.barplot(data=class_metrics_df, x='class', y=metric)
        plt.title(f'Class-Specific {metric.capitalize()}')
        plt.xlabel('Class')
        plt.ylabel(metric.capitalize())
        plt.xticks(rotation=45, ha="right")
        plt.tight_layout()
        plt.show()

    # Create a new DataFrame that includes the F1 scores and the class occurrences
    class_f1_scores = class_metrics_df.groupby('class')['f1'].mean().reindex(class_occurrences_dict.keys())
    combined_df = pd.DataFrame({
        'class': list(class_occurrences_dict.keys()),
        'occurrences': list(class_occurrences_dict.values()),
        'f1_score': class_f1_scores
    })

    # Plot for Class Occurrences with F1 scores
    fig, ax1 = plt.subplots(figsize=(12, 6))

    # Plot occurrences on the left y-axis
    ax1.bar(combined_df['class'], combined_df['occurrences'], color="lightblue", label="Occurrences")
    ax1.set_xlabel('Class', fontsize=14)
    ax1.set_ylabel('Number of Occurrences', fontsize=14, color="blue")
    ax1.tick_params(axis='y', labelcolor="blue")
    plt.xticks(rotation=45, ha="right")

    # Plot F1 scores on the right y-axis
    ax2 = ax1.twinx()
    ax2.plot(combined_df['class'], combined_df['f1_score'], color="red", marker='o', label="F1 Score")
    ax2.set_ylabel('F1 Score', fontsize=14, color="red")
    ax2.tick_params(axis='y', labelcolor="red")

    # Add title
    plt.title('Number of Times Each Class Occurs with Corresponding F1 Score', fontsize=16)

    plt.tight_layout()
    plt.show()

# Example usage:
# plot_metrics_with_class_occurrences_and_f1(general_metrics_df, class_metrics_df, relation_type_counts)

In [ ]:
# Assuming you have general_metrics_df and class_metrics_df already created
plot_metrics(general_metrics_df, class_metrics_df)

In [34]:
def average_metrics(score_list):
    # Initialize a dictionary to hold the sum of all metrics
    avg_metrics = {
        'micro_f1': 0,
        'micro_precision': 0,
        'micro_recall': 0,
        'macro_f1': 0,
        'macro_precision': 0,
        'macro_recall': 0,
        'accuracy': 0,
        'class_metrics': {}
    }
    
    if not score_list:  # Check if the list is empty
        return avg_metrics
    
    # First, we gather all classes across all score dictionaries
    all_classes = set()
    for scores in score_list:
        all_classes.update(scores['class_metrics'].keys())
    
    # Initialize the class_metrics to sum the values for each class
    for cls in all_classes:
        avg_metrics['class_metrics'][cls] = {
            'f1': 0,
            'precision': 0,
            'recall': 0,
            'accuracy': 0
        }
    
    # Accumulate metrics across all score dictionaries
    for scores in score_list:
        avg_metrics['micro_f1'] += scores['micro_f1']
        avg_metrics['micro_precision'] += scores['micro_precision']
        avg_metrics['micro_recall'] += scores['micro_recall']
        avg_metrics['macro_f1'] += scores['macro_f1']
        avg_metrics['macro_precision'] += scores['macro_precision']
        avg_metrics['macro_recall'] += scores['macro_recall']
        avg_metrics['accuracy'] += scores['accuracy']
        
        # Accumulate class metrics
        for cls, metrics in scores['class_metrics'].items():
            avg_metrics['class_metrics'][cls]['f1'] += metrics['f1']
            avg_metrics['class_metrics'][cls]['precision'] += metrics['precision']
            avg_metrics['class_metrics'][cls]['recall'] += metrics['recall']
            avg_metrics['class_metrics'][cls]['accuracy'] += metrics['accuracy']
    
    # Calculate averages
    num_scores = len(score_list)
    
    if num_scores > 0:  # Prevent division by zero
        avg_metrics['micro_f1'] /= num_scores
        avg_metrics['micro_precision'] /= num_scores
        avg_metrics['micro_recall'] /= num_scores
        avg_metrics['macro_f1'] /= num_scores
        avg_metrics['macro_precision'] /= num_scores
        avg_metrics['macro_recall'] /= num_scores
        avg_metrics['accuracy'] /= num_scores
    
        # Calculate per-class averages
        for cls in avg_metrics['class_metrics']:
            avg_metrics['class_metrics'][cls]['f1'] /= num_scores
            avg_metrics['class_metrics'][cls]['precision'] /= num_scores
            avg_metrics['class_metrics'][cls]['recall'] /= num_scores
            avg_metrics['class_metrics'][cls]['accuracy'] /= num_scores
    
    return avg_metrics

In [ ]:
average_metrics(scores_rels)

In [ ]:
# Function to save the data (scores_rels, general_metrics_df, class_metrics_df) with custom name
def save_data(scores_rels, general_metrics_df, class_metrics_df, label_predictions, incorrect_format, folder_path, name):
    # Ensure the folder exists
    os.makedirs(folder_path, exist_ok=True)
    
    # Save scores_rels as a pickle file
    scores_rels_filepath = os.path.join(folder_path, f"dspy_{name}_scores_rels.pkl")
    with open(scores_rels_filepath, "wb") as handle:
        pickle.dump(scores_rels, handle)
    
    # Save general_metrics_df as a pickle file
    general_metrics_filepath = os.path.join(folder_path, f"dspy_{name}_general_metrics_df.pkl")
    general_metrics_df.to_pickle(general_metrics_filepath)
    
    # Save class_metrics_df as a pickle file
    class_metrics_filepath = os.path.join(folder_path, f"dspy_{name}_general_class_metrics_df.pkl")
    class_metrics_df.to_pickle(class_metrics_filepath)

    # save label_predictions as a pickle file
    label_predictions_filepath= os.path.join(folder_path, f"dspy_{name}_label_predictions.pkl")
    with open(label_predictions_filepath, "wb") as f:
        pickle.dump(label_predictions, f)
    
    # save incorrect_format as a pickle file
    incorrect_format_filepath = os.path.join(folder_path, f"dspy_{name}_incorrect_format.pkl")
    with open(incorrect_format_filepath,"wb") as f:
        pickle.dump(incorrect_format,f)
    
    # Return the paths of the saved files
    return scores_rels_filepath, general_metrics_filepath, class_metrics_filepath, label_predictions_filepath, incorrect_format_filepath

# Example usage:
# scores_rels, general_metrics_df, class_metrics_df are assumed to be your datasets
save_data(scores_rels, general_metrics_df, class_metrics_df, label_predictions,incorrect_format,"/work3/s174159/LLM_Thesis/plots/data", "dspy_0_shot")

In [42]:

def relation_f1_score(expected, predicted, trace=None):
    def parse_input(input_data):
        if isinstance(input_data, str):
            try:
                # Try to parse as JSON
                return json.loads(input_data)
            except json.JSONDecodeError:
                try:
                    # Fallback to ast.literal_eval
                    return ast.literal_eval(input_data)
                except (ValueError, SyntaxError):
                    # Return None if both parsing attempts fail
                    return None
        elif isinstance(input_data, dict):
            # Wrap dict in a list
            return [input_data]
        return input_data

    # Parse expected and predicted inputs
    expected = parse_input(expected)
    predicted = parse_input(predicted)

    # print("Predicted", predicted)



    expected_relations = set()

    
    entities = expected.entities
    relations = expected.relations
    
    flat_entities = [entity for sublist in entities for entity in sublist]
    for rel in relations:
        entity1_name = flat_entities[rel['h']]['name']
        entity2_name = flat_entities[rel['t']]['name']
        relation_type = rel['r']
        expected_relations.add((entity1_name, entity2_name, relation_type))
    
    # print("expected relations", expected_relations)
    
    # Extract predicted relations
    predicted_relations = {
        (rel['entity1'], rel['entity2'], rel['relation'])
        for rel in predicted
    }


    # Create binary labels for true/false positives
    y_true = [1 if rel in expected_relations else 0 for rel in predicted_relations]
    y_pred = [1] * len(predicted_relations)  # All predicted relations are labeled as 1

    # Handle case where there are no true relations in expected
    if not y_true:
        return 0

    # Calculate F1 Score
    f1 = f1_score(y_true, y_pred, zero_division=0)
    if not f1:
        return 0
    return f1 #,expected_relations, predicted_relations



# GPT4oMINI

In [90]:
dspy.settings.configure(lm=gpt4)

## 3

In [87]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

In [ ]:
tp3 = BootstrapFewShotWithRandomSearch(metric=relation_f1_score, max_bootstrapped_demos=2, max_labeled_demos=1, num_candidate_programs=5, num_threads=8, max_rounds = 1)
optimized_program_3 = tp3.compile(rel_extractor, trainset=dataset, valset=dev_data, restrict=range(10))

In [ ]:
optimized_program_3.save("/work3/s174159/LLM_Thesis/prompt/opt_3.json")

In [46]:
optimized_program_3 = rel_extractor
optimized_program_3.load("/work3/s174159/LLM_Thesis/prompt/opt_3.json")

In [ ]:
scores_rels,label_predictions,incorrect_format = get_score(optimized_program_3,test_data)

In [ ]:
# Assuming you have general_metrics_df and class_metrics_df already created
general_metrics_df, class_metrics_df = sort_data(scores_rels)
plot_metrics(general_metrics_df, class_metrics_df)
average_metrics(scores_rels)

In [ ]:
save_data(scores_rels, general_metrics_df, class_metrics_df,label_predictions,incorrect_format, "/work3/s174159/LLM_Thesis/plots/data", "3_shot")

## 6

In [ ]:
tp6 = BootstrapFewShotWithRandomSearch(metric=relation_f1_score, max_bootstrapped_demos=6, max_labeled_demos=3, num_candidate_programs=5, num_threads=8, max_rounds = 1)
optimized_program_6 = tp6.compile(rel_extractor, trainset=dataset, valset=dev_data, restrict=range(10))

In [ ]:
optimized_program_6.save("/work3/s174159/LLM_Thesis/prompt/opt_6.json")

In [ ]:
scores_rels,label_predictions,incorrect_format = get_score(optimized_program_6,test_data)

In [ ]:
scores_rels

In [ ]:
# Assuming you have general_metrics_df and class_metrics_df already created
general_metrics_df, class_metrics_df = sort_data(scores_rels)
plot_metrics(general_metrics_df, class_metrics_df)
average_metrics(scores_rels)

save_data(scores_rels, general_metrics_df, class_metrics_df,label_predictions,incorrect_format, "/work3/s174159/LLM_Thesis/plots/data", "6_shot")

## 10

In [ ]:
tp10 = BootstrapFewShotWithRandomSearch(metric=relation_f1_score, max_bootstrapped_demos=10, max_labeled_demos=5, num_candidate_programs=5, num_threads=8, max_rounds = 1)
optimized_program_10 = tp10.compile(rel_extractor, trainset=dataset, valset=dev_data, restrict=range(10))

In [ ]:
optimized_program_6.save("/work3/s174159/LLM_Thesis/prompt/opt_6.json")

In [ ]:
scores_rels,label_predictions,incorrect_format  = get_score(optimized_program_10,test_data)

In [ ]:
# Assuming you have general_metrics_df and class_metrics_df already created
general_metrics_df, class_metrics_df = sort_data(scores_rels)
plot_metrics(general_metrics_df, class_metrics_df)
average_metrics(scores_rels)

save_data(scores_rels, general_metrics_df, class_metrics_df,label_predictions,incorrect_format, "/work3/s174159/LLM_Thesis/plots/data", "10_shot")

## GPT4

In [93]:
dspy.settings.configure(lm=gpt4)

## 3

In [94]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

In [ ]:
gtp3 = BootstrapFewShotWithRandomSearch(metric=relation_f1_score, max_bootstrapped_demos=2, max_labeled_demos=1, num_candidate_programs=5, num_threads=8, max_rounds = 1)
goptimized_program_3 = gtp3.compile(rel_extractor, trainset=dataset, valset=dev_data, restrict=range(10))

In [ ]:
goptimized_program_3.save("/work3/s174159/LLM_Thesis/prompt/gopt_3.json")

In [ ]:
goptimized_program_3 = rel_extractor
goptimized_program_3.load("/work3/s174159/LLM_Thesis/prompt/gopt_3.json")

In [ ]:
scores_rels,label_predictions,incorrect_format = get_score(goptimized_program_3,test_data)

In [ ]:
# Assuming you have general_metrics_df and class_metrics_df already created
general_metrics_df, class_metrics_df = sort_data(scores_rels)
plot_metrics(general_metrics_df, class_metrics_df)
average_metrics(scores_rels)

In [ ]:
save_data(scores_rels, general_metrics_df, class_metrics_df,label_predictions,incorrect_format, "/work3/s174159/LLM_Thesis/plots/data", "g3_shot")

## 6

In [ ]:
gtp6 = BootstrapFewShotWithRandomSearch(metric=relation_f1_score, max_bootstrapped_demos=6, max_labeled_demos=3, num_candidate_programs=5, num_threads=8, max_rounds = 1)
goptimized_program_6 = gtp6.compile(rel_extractor, trainset=dataset, valset=dev_data, restrict=range(10))

In [ ]:
goptimized_program_6.save("/work3/s174159/LLM_Thesis/prompt/gopt_6.json")

In [ ]:
scores_rels,label_predictions,incorrect_format = get_score(goptimized_program_6,test_data)

In [ ]:
scores_rels

In [ ]:
# Assuming you have general_metrics_df and class_metrics_df already created
general_metrics_df, class_metrics_df = sort_data(scores_rels)
plot_metrics(general_metrics_df, class_metrics_df)
average_metrics(scores_rels)

save_data(scores_rels, general_metrics_df, class_metrics_df,label_predictions,incorrect_format, "/work3/s174159/LLM_Thesis/plots/data", "g6_shot")

## 10

In [ ]:
gtp10 = BootstrapFewShotWithRandomSearch(metric=relation_f1_score, max_bootstrapped_demos=10, max_labeled_demos=5, num_candidate_programs=5, num_threads=8, max_rounds = 1)
goptimized_program_10 = gtp10.compile(rel_extractor, trainset=dataset, valset=dev_data, restrict=range(10))

In [ ]:
optimized_program_6.save("/work3/s174159/LLM_Thesis/prompt/gopt_6.json")

In [ ]:
scores_rels,label_predictions,incorrect_format  = get_score(goptimized_program_10,test_data)

In [ ]:
# Assuming you have general_metrics_df and class_metrics_df already created
general_metrics_df, class_metrics_df = sort_data(scores_rels)
plot_metrics(general_metrics_df, class_metrics_df)
average_metrics(scores_rels)

save_data(scores_rels, general_metrics_df, class_metrics_df,label_predictions,incorrect_format, "/work3/s174159/LLM_Thesis/plots/data", "g10_shot")

## Local models

In [ ]:
lm = dspy.HFClientVLLM(model="meta-llama/Llama-2-70b-chat-hf", port=8000, url="http://localhost")

In [ ]:
gtp3 = BootstrapFewShotWithRandomSearch(metric=relation_f1_score, max_bootstrapped_demos=2, max_labeled_demos=1, num_candidate_programs=5, num_threads=8, max_rounds = 1)
goptimized_program_3 = gtp3.compile(rel_extractor, trainset=dataset, valset=dev_data, restrict=range(10))